# Import the required libraries

In [81]:
import pinecone
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key="")
pinecone.init(api_key="", environment="eastus-azure")

index = pinecone.Index('tipcraft')


# Load dataset

In [49]:
# Load your dataset
user_data = pd.read_csv('C:\\Users\\kasun\\Downloads\\TipCraftHub\\notebooks\\user_with_clusters_final.csv')
reviews = pd.read_csv('C:\\Users\\kasun\\Downloads\\yelp\\data\\yelp_reviews.csv')
tips = pd.read_csv('C:\\Users\\kasun\\Downloads\\yelp\\data\\yelp_tips.csv')

# User ID and Shop ID that you want to search

In [58]:
user_id_to_search = "nmdkHL2JKFx55T3nq5VziA"
shop_id_to_search = "VQcCL9PiNL_wkGf-uF3fjg"

# Global variables

In [59]:
userVector = ''
userCluster = ''

# Function to get the vector representation and cluster for a user ID

In [60]:
def get_user_vector_by_id(user_data, user_id):
    global userVector
    # Search for a user by ID
    user_info = user_data[user_data['user_id'] == user_id]

    # Check if the user exists
    if not user_info.empty:
        # Get the vector representation from the 'vector' column
        vector_representation = user_info['representations_as_vectors_auto'].values[0]
        dense_vector = [float(value) for value in vector_representation[1:-1].split(',')]
        # Get the cluster from the 'cluster' column
        cluster = user_info['cluster_auto'].values[0]
        userVector = dense_vector
        return userVector, cluster
    else:
        print(f"User with ID {user_id} not found.")
        return None


user_vector = get_user_vector_by_id(user_data, user_id_to_search)

if user_vector is not None:
    print(f"Vector representation for user {user_id_to_search}: {user_vector[0]}"
          f"\nCluster for user {user_id_to_search}: {user_vector[1]}")
    userCluster = user_vector[1]


Vector representation for user nmdkHL2JKFx55T3nq5VziA: [101.00413645823296, 89.16827467461427, 70.97300827618069]
Cluster for user nmdkHL2JKFx55T3nq5VziA: 2


# Search for similar users on Pinecone Vector Index

In [63]:
def getUser(id):
    # Query Pinecone index
    index = pinecone.Index("tipcraft")

    # Perform a similarity search
    namespace = "cluster_auto"
    metadata_filter = {"model_name": "cluster_auto", 'cluster': int(userCluster)}
    response = index.query(
        vector=userVector,
        top_k=100,
        include_values=False,
        include_metadata=True,
        filter=metadata_filter,
        namespace=namespace
    )
    
    # Extract user IDs from the response
    user_ids = [match['id'] for match in response['matches'] if match['id'] != id]
    return user_ids


searched_user_id = "qVc8ODYU5SZjKXVBgXdI7w"
result_user_ids = getUser(searched_user_id)
print("Top user IDs:", result_user_ids)


Top user IDs: ['nmdkHL2JKFx55T3nq5VziA', '2l0O1EI1m0yWjFo2zSt71w', 'GHoG4X4FY8D8L563zzPX5w', 'om5ZiponkpRqUNa3pVPiRg', 'HFECrzYDpgbS5EmTBtj2zQ', 'eSlOI3GhroEtcbaD_nFXJQ', 'WWWujmZ95y0a2RR5_kQ9sw', 'ysCBsXWPB-LAiewVS3jZfQ', 'U4INQZOPSUaj8hMjLlZ3KA', 'nzsv-p1O8gCfP3XijfQrIw', 'JADdo9NEeO5Az9aOYbyvZA', 'JnGtgOPpkjyWOvWM0SYEXg', '0zDHiDbj79uBOy3OAcE25A', 'IwHLuykqqmGVNHJ_UCwXWg', 'PKEzKWv_FktMm2mGPjwd0Q', 'pou3BbKsIozfH50rxmnMew', '5BpJEC8YVi80ecWT5CtlfQ', 'HXOfPXPMvdXNpQ_l0RRjKw', 'oyXIV-9GRBBZrFydPCaR2w', 'lConjPFTZqz4zWELK_EJtw', 'gjhzKWsqCIrpEd9pevbKZw', 'Lu8y2ayxVlAzuJEVfVcH4w', 'I-FmdvI28N7OIIt4JZOgIg', 'tgeFUChlh7v8bZFVl2-hjQ', 'whINg-cC-FiAv_ATDGMDTg', 'AHRrG3T1gJpHvtpZ-K0G_g', 'IDVFG1pNSHIHoVuoLuZpcQ', 'WWnhqRnWWjutMqh-2SzEuQ', '6ce1NSAUt1Kr7efrLI8XQA', 'bQCHF5rn5lMI9c5kEwCaNA', '8k3aO-mPeyhbR5HUucA5aA', 'sOzeK0dE0BYx4Mc8dmoenQ', 'PkNs-Rw6MHQfu6j5pBZMMw', 'cVrM2BvOvPV0Vi_rC8kepQ', 'dIIKEfOgo0KqUfGQvGikPg', 'lYp818T-xh8Ss79To5Azhw', '6s-g2vFu12OemhiK3FJuOQ', 'CQUDh80m48xnzUkx-X5NAw

# Get the reviews and tips from similar users for the shop

In [73]:
content = []

for user_id_to_search in result_user_ids:
    # Filter reviews for the specific user and shop
    user_reviews = reviews[(reviews['user_id'] == user_id_to_search) & (reviews['business_id'] == shop_id_to_search)]

    # Filter tips for the specific user and shop
    user_tips = tips[(tips['user_id'] == user_id_to_search) & (tips['business_id'] == shop_id_to_search)]

    reviews_string = "\n".join(user_reviews['text']) if not user_reviews.empty else ''
    tips_string = "\n".join(user_tips['text']) if not user_tips.empty else ''

    # Display the results
    print(f"\nUser ID: {user_id_to_search}")
    print("User Reviews:")
    print(reviews_string)

    print("\nUser Tips:")
    print(tips_string)

    # Add reviews and tips to the content list if they are not empty
    if reviews_string:
        content.append(reviews_string)
    if tips_string:
        content.append(tips_string)

print("\nCombined Content:")
print("\n".join(content))




User ID: nmdkHL2JKFx55T3nq5VziA
User Reviews:


User Tips:

User ID: 2l0O1EI1m0yWjFo2zSt71w
User Reviews:


User Tips:

User ID: GHoG4X4FY8D8L563zzPX5w
User Reviews:


User Tips:

User ID: om5ZiponkpRqUNa3pVPiRg
User Reviews:


User Tips:

User ID: HFECrzYDpgbS5EmTBtj2zQ
User Reviews:


User Tips:

User ID: eSlOI3GhroEtcbaD_nFXJQ
User Reviews:


User Tips:

User ID: WWWujmZ95y0a2RR5_kQ9sw
User Reviews:


User Tips:

User ID: ysCBsXWPB-LAiewVS3jZfQ
User Reviews:


User Tips:

User ID: U4INQZOPSUaj8hMjLlZ3KA
User Reviews:


User Tips:

User ID: nzsv-p1O8gCfP3XijfQrIw
User Reviews:
We only ordered 3 items here as it was our 2nd dinner of the night. The gumbo -- I was on a mission to try every gumbo in the city -- was disappointing, very watery and no flavor at all. was made with a light roux instead of a darker one.

The fried alligator was grisly, but if you managed to find a piece that wasn't grisly, it was okay. Nothing really special.

The redfish beignets though was our favorite. Re

In [75]:
print(content)

["We only ordered 3 items here as it was our 2nd dinner of the night. The gumbo -- I was on a mission to try every gumbo in the city -- was disappointing, very watery and no flavor at all. was made with a light roux instead of a darker one.\n\nThe fried alligator was grisly, but if you managed to find a piece that wasn't grisly, it was okay. Nothing really special.\n\nThe redfish beignets though was our favorite. Really delicious drizzled in some sort of a honey sauce. I would just come in to eat these!\n\nService is good, our waitress was very friendly and didn't seem to mind that we were only ordering a few things and drinking water.", "Why 5 stars? Because it satisfied my NOLA cajun/seafood/oyster craving!\n\n   ---   ---   ---\n\nSo I had to grab oysters while in New Orleans right? Royal House... bravo. \n\nThe definite highlight were actually the crab claws. Absolutely out of this world. One of the best things I've ever eaten. It would be a sin to not order this on a trip to Royal

# Function to ask a question and perform RAG

In [78]:
# Function to ask a question and perform RAG
def ask_question_rag(question):
        rag_response= client.chat.completions.create(
            model="gpt-4-1106-preview",#gpt-4-1106-preview
            messages=[
                {"role": "system", "content": f"Imagine yourself as a seasoned tip generator for Yelp.com. Your role is to provide personalized tips for users planning to visit specific shops. Your goal is to offer 3 to 6 concise and valuable tips that enhance the user's experience during their shop visit."
                f"Consider the importance of each tip, ensuring they are not only short and useful but also tailored to the specific characteristics of the shops in question. To achieve this, analyze the provided information, including previous reviews and tips from individuals with similar preferences and behaviors."
                f"Your task is to extract insights from these reviews and craft tips that resonate with the potential visitors. By delving into the nuances of the previous user reviews and tips, you can offer actionable advice that aligns with their expectations."
                f"Remember, the key is to deliver the most useful tips that cater to the unique aspects of each shop. The success of your suggestions lies in their relevance to the users' needs and preferences."
                f"[Note: Please incorporate the provided reviews and tips in your analysis.] "
                f" Past Reviews and Tips for this shop = [ {content} ]"},
                {"role": "user", "content": question},
            ]
        )
        print(str(rag_response.choices[0].message.content))


ask_question_rag('I\'m planning to visit this shop. What are some tips?')



Certainly! Here are some personalized tips for your visit to the shop:

1. **A Must Try**: Don’t miss out on the crab claws and the charbroiled oysters. They have received high praise from past visitors and seem to be the crowd favorites!

2. **Ordering Strategy**: If you’re aiming for a flavorful experience, you might want to skip the gumbo and instead go for the lauded redfish beignets. The savory honey sauce is said to be delightful.

3. **Seating Choice**: For a memorable experience, request a balcony seat. It offers great people-watching opportunities and a view of the local street performances, adding to the ambiance of your dining experience.

4. **Mind the Salt**: Some guests found the dishes to be on the saltier side. Don't hesitate to ask the kitchen to go easy on the salt if you’re sensitive to it.

5. **Sharing is Caring**: Portions like the fried alligator bites might be smaller than expected. Consider this if you plan to share dishes, and maybe order an extra item or two.